In [ ]:
import pathlib as pb
import json

import polars as pl
import polars_st as st
import marimo as mo
import folium as fl

import gtfs_kit_polars as gk


HERE = pb.Path(__file__).resolve().parent if "__file__" in globals() else pb.Path.cwd()
PROJECT_ROOT = HERE.parent  # notebooks/ -> project/
DATA = (PROJECT_ROOT / "data").resolve()

# Notes

- All outputs are LazyFrames, so we need to collect them to display them here

In [ ]:
# List feed

gk.list_feed(DATA / "cairns_gtfs.zip").collect()

file_name,file_size
str,i64
"""calendar_dates.txt""",387
"""routes.txt""",1478
"""trips.txt""",143081
"""stops.txt""",26183
"""agency.txt""",199
"""stop_times.txt""",2561019
"""shapes.txt""",864694
"""calendar.txt""",337


In [ ]:
# Read feed and describe

feed = gk.read_feed(DATA / "cairns_gtfs.zip", dist_units="m")
feed.describe().collect()

indicator,value
str,str
"""agencies""","""['Department of Transport and …"
"""timezone""","""Australia/Brisbane"""
"""start_date""","""20140526"""
"""end_date""","""20141228"""
"""num_routes""","""22"""
…,…
"""num_shapes""","""54"""
"""sample_date""","""20140529"""
"""num_routes_active_on_sample_da…","""20"""


In [ ]:
mo.output.append(feed.stop_times.head().collect())
feed_1 = feed.append_dist_to_stop_times()
mo.output.append(feed_1.stop_times.head().collect())

shape: (5, 7) trip_id arrival_time departure_time stop_id stop_sequence pickup_type drop_off_type str str str str i32 i8 i8 "CNS2014-CNS_MUL-Weekday-00-416… "05:50:00" "05:50:00" "750337" 1 0 0 "CNS2014-CNS_MUL-Weekday-00-416… "05:50:00" "05:50:00" "750000" 2 0 0 "CNS2014-CNS_MUL-Weekday-00-416… "05:52:00" "05:52:00" "750001" 3 0 0 "CNS2014-CNS_MUL-Weekday-00-416… "05:54:00" "05:54:00" "750002" 4 0 0 "CNS2014-CNS_MUL-Weekday-00-416… "05:55:00" "05:55:00" "750003" 5 0 0 
 shape: (5, 8) trip_id arrival_time departure_time stop_id stop_sequence pickup_type drop_off_type shape_dist_traveled str str str str i32 i8 i8 f64 "CNS2014-CNS_MUL-Saturday-00-41… "06:16:00" "06:16:00" "750337" 1 0 0 0.0 "CNS2014-CNS_MUL-Saturday-00-41… "06:16:00" "06:16:00" "750000" 2 0 0 468.64031 "CNS2014-CNS_MUL-Saturday-00-41… "06:18:00" "06:18:00" "750001" 3 0 0 1190.379646 "CNS2014-CNS_MUL-Saturday-00-41… "06:20:00" "06:20:00" "750002" 4 0 0 2154.78462 "CNS2014-CNS_MUL-Saturday-00-41… "06:21:00" "06:21:00" "750003" 5 0 0 2619.002693

In [ ]:
week = feed_1.get_first_week()
dates = [week[0], week[6]]
dates

In [ ]:
# Trip stats; reuse these for later speed ups

trip_stats = feed_1.compute_trip_stats().collect()
trip_stats

trip_id,route_id,route_short_name,route_type,direction_id,shape_id,stop_pattern_name,num_stops,start_time,end_time,start_stop_id,end_stop_id,duration,is_loop,distance,speed
str,str,str,i8,i8,str,str,u32,str,str,str,str,f64,bool,f64,f64
"""CNS2014-CNS_MUL-Weekday-00-416…","""110-423""","""110""",3,0,"""1100023""","""0-1""",35,"""05:50:00""","""06:50:00""","""750337""","""750449""",1.0,false,32.507121,32.507121
"""CNS2014-CNS_MUL-Saturday-00-41…","""110-423""","""110""",3,0,"""1100023""","""0-1""",35,"""06:16:00""","""07:10:00""","""750337""","""750449""",0.9,false,32.507121,36.119024
"""CNS2014-CNS_MUL-Weekday-00-416…","""110-423""","""110""",3,0,"""1100023""","""0-1""",35,"""06:20:00""","""07:20:00""","""750337""","""750449""",1.0,false,32.507121,32.507121
"""CNS2014-CNS_MUL-Weekday-00-416…","""110-423""","""110""",3,0,"""1100023""","""0-1""",35,"""06:50:00""","""07:50:00""","""750337""","""750449""",1.0,false,32.507121,32.507121
"""CNS2014-CNS_MUL-Weekday-00-416…","""110-423""","""110""",3,0,"""1100023""","""0-1""",35,"""07:15:00""","""08:20:00""","""750337""","""750449""",1.083333,false,32.507121,30.006574
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CNS2014-CNS_MUL-Weekday-00-417…","""150E-423""","""150E""",3,1,"""150E0009""","""1-1""",40,"""20:15:00""","""21:22:00""","""750453""","""750412""",1.116667,false,38.423999,34.409551
"""CNS2014-CNS_MUL-Sunday-00-4180…","""150E-423""","""150E""",3,1,"""150E0008""","""1-1""",40,"""21:00:00""","""22:09:00""","""750453""","""750412""",1.15,false,38.139859,33.165095
"""CNS2014-CNS_MUL-Saturday-00-41…","""150E-423""","""150E""",3,1,"""150E0009""","""1-1""",40,"""21:13:00""","""22:22:00""","""750453""","""750412""",1.15,false,38.423999,33.412173


In [ ]:
# Pass in trip stats to avoid recomputing them

network_stats = feed_1.compute_network_stats(dates).collect()
network_stats

date,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time
str,u32,u32,u32,u32,u32,f64,f64,f64,i32,str,str
"""20140526""",416,20,622,622,622,13774.030583,472.6,29.145219,39,"""08:16:00""","""08:18:00"""
"""20140601""",411,14,266,266,266,6390.847266,197.683333,32.328711,17,"""14:31:00""","""14:37:00"""


In [ ]:
nts = feed_1.compute_network_time_series(dates, num_minutes=6 * 60).collect()
nts

datetime,num_trip_starts,num_trip_ends,num_trips,service_distance,service_duration,service_speed
datetime[μs],f64,f64,f64,f64,f64,f64
2014-05-26 00:00:00,3.0,0.0,3.0,66.182329,2.033333,32.548687
2014-05-26 06:00:00,240.0,214.0,242.0,4954.226378,173.8,28.50533
2014-05-26 12:00:00,253.0,248.0,281.0,5505.33739,193.216667,28.493077
2014-05-26 18:00:00,126.0,155.0,160.0,3248.284486,103.55,31.369237
2014-06-01 00:00:00,0.0,0.0,0.0,26.198808,0.766667,34.172358
2014-06-01 06:00:00,80.0,68.0,80.0,1794.087434,55.766667,32.171323
2014-06-01 12:00:00,110.0,111.0,122.0,2615.694171,81.516667,32.087845
2014-06-01 18:00:00,76.0,84.0,87.0,1954.866853,59.633333,32.781445


In [ ]:
gk.downsample(nts, num_minutes=12 * 60).collect()

datetime,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed
datetime[μs],f64,f64,f64,f64,f64,f64
2014-05-26 00:00:00,242.0,243.0,214.0,5020.408707,175.833333,28.552087
2014-05-26 12:00:00,408.0,379.0,403.0,8753.621876,296.766667,29.496648
2014-06-01 00:00:00,80.0,80.0,68.0,1820.286242,56.533333,32.198459
2014-06-01 12:00:00,198.0,186.0,195.0,4570.561024,141.15,32.380879


In [ ]:
# Stop time series
stop_ids = feed.stops.head(1).collect()["stop_id"].to_list()
sts = feed_1.compute_stop_time_series(
    dates, stop_ids=stop_ids, num_minutes=12 * 60
).collect()
sts

datetime,stop_id,num_trips
datetime[μs],str,i32
2014-05-26 00:00:00,"""750000""",13
2014-05-26 12:00:00,"""750000""",17
2014-06-01 00:00:00,"""750000""",5
2014-06-01 12:00:00,"""750000""",11


In [ ]:
gk.downsample(sts, num_minutes=24 * 60).collect()

datetime,stop_id,num_trips
datetime[μs],str,i32
2014-05-26 00:00:00,"""750000""",30
2014-06-01 00:00:00,"""750000""",16


In [ ]:
# Route time series

rts = feed_1.compute_route_time_series(dates, num_minutes=12 * 60).collect()
rts

datetime,route_id,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed
datetime[μs],str,f64,f64,f64,f64,f64,f64
2014-05-26 00:00:00,"""110-423""",23.0,23.0,19.0,677.652816,21.016667,32.243592
2014-05-26 00:00:00,"""111-423""",22.0,22.0,18.0,713.781443,21.583333,33.070955
2014-05-26 00:00:00,"""112-423""",5.0,5.0,4.0,87.584926,2.483333,35.269098
2014-05-26 00:00:00,"""113-423""",3.0,3.0,3.0,74.298371,2.083333,35.663218
2014-05-26 00:00:00,"""120-423""",12.0,12.0,11.0,323.598592,9.583333,33.76681
…,…,…,…,…,…,…,…
2014-06-01 12:00:00,"""131N-423""",1.0,1.0,1.0,12.32682,0.483333,25.503766
2014-06-01 12:00:00,"""133-423""",12.0,11.0,12.0,181.92692,6.516667,27.917174
2014-06-01 12:00:00,"""142-423""",6.0,6.0,6.0,143.631505,5.1,28.16304


In [ ]:
# Route timetable

route_id = feed_1.routes.head(1).collect()["route_id"].item(0)
feed_1.build_route_timetable(route_id, dates).collect()

trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,date
str,str,str,str,i32,i8,i8,f64,str,str,str,i8,str,str,str
"""CNS2014-CNS_MUL-Weekday-00-416…","""05:50:00""","""05:50:00""","""750337""",1,0,0,0.0,"""110-423""","""CNS2014-CNS_MUL-Weekday-00""","""The Pier Cairns Terminus""",0,null,"""1100023""","""20140526"""
"""CNS2014-CNS_MUL-Weekday-00-416…","""05:50:00""","""05:50:00""","""750000""",2,0,0,468.64031,"""110-423""","""CNS2014-CNS_MUL-Weekday-00""","""The Pier Cairns Terminus""",0,null,"""1100023""","""20140526"""
"""CNS2014-CNS_MUL-Weekday-00-416…","""05:52:00""","""05:52:00""","""750001""",3,0,0,1190.379646,"""110-423""","""CNS2014-CNS_MUL-Weekday-00""","""The Pier Cairns Terminus""",0,null,"""1100023""","""20140526"""
"""CNS2014-CNS_MUL-Weekday-00-416…","""05:54:00""","""05:54:00""","""750002""",4,0,0,2154.78462,"""110-423""","""CNS2014-CNS_MUL-Weekday-00""","""The Pier Cairns Terminus""",0,null,"""1100023""","""20140526"""
"""CNS2014-CNS_MUL-Weekday-00-416…","""05:55:00""","""05:55:00""","""750003""",5,0,0,2619.002693,"""110-423""","""CNS2014-CNS_MUL-Weekday-00""","""The Pier Cairns Terminus""",0,null,"""1100023""","""20140526"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CNS2014-CNS_MUL-Sunday-00-4166…","""23:56:00""","""23:56:00""","""750038""",28,0,0,27827.737141,"""110-423""","""CNS2014-CNS_MUL-Sunday-00""","""Palm Cove""",1,null,"""1100016""","""20140601"""
"""CNS2014-CNS_MUL-Sunday-00-4166…","""23:59:00""","""23:59:00""","""750339""",29,0,0,29533.971877,"""110-423""","""CNS2014-CNS_MUL-Sunday-00""","""Palm Cove""",1,null,"""1100016""","""20140601"""
"""CNS2014-CNS_MUL-Sunday-00-4166…","""24:01:00""","""24:01:00""","""750039""",30,0,0,30445.639081,"""110-423""","""CNS2014-CNS_MUL-Sunday-00""","""Palm Cove""",1,null,"""1100016""","""20140601"""


In [ ]:
# Locate trips

loc = feed_1.locate_trips(dates[0], times=["08:00:00", "09:00:00"]).collect()
loc

trip_id,shape_id,route_id,direction_id,time,rel_dist,lon,lat
str,str,str,i8,str,f64,f64,f64
"""CNS2014-CNS_MUL-Weekday-00-418…","""1500020""","""150-423""",0,"""09:00:00""",1.0,145.779299,-16.920767
"""CNS2014-CNS_MUL-Weekday-00-417…","""1300017""","""130-423""",1,"""08:00:00""",0.975635,145.739956,-16.925606
"""CNS2014-CNS_MUL-Weekday-00-416…","""1100023""","""110-423""",0,"""09:00:00""",0.152945,145.669459,-16.762864
"""CNS2014-CNS_MUL-Weekday-00-416…","""1210012""","""121-423""",0,"""08:00:00""",0.496428,145.733331,-16.87531
"""CNS2014-CNS_MUL-Weekday-00-418…","""1430027""","""143-423""",1,"""09:00:00""",0.239443,145.753483,-16.933478
…,…,…,…,…,…,…,…
"""CNS2014-CNS_MUL-Weekday-00-416…","""1100024""","""110-423""",1,"""09:00:00""",0.878124,145.663138,-16.758061
"""CNS2014-CNS_MUL-Weekday-00-417…","""1410016""","""141-423""",0,"""08:00:00""",0.221598,145.728196,-16.961232
"""CNS2014-CNS_MUL-Weekday-00-416…","""1210013""","""121-423""",1,"""09:00:00""",1.0,145.692973,-16.906822


In [ ]:
# Map routes

rsns = feed_1.routes.head().collect()["route_short_name"].to_list()[2:4]
feed_1.map_routes(route_short_names=rsns, show_stops=True)

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_d9aa7060d08aa06cbaa58663d314787a {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>

 <style>html, body {
 width: 100%;
 height: 100%;
 margin: 0;
 padding: 0;
 }
 </style>

 <style>#map {
 position:absolute;
 top:0;
 bottom:0;
 right:0;
 left:0;
 }
 </style>

 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>

 
</head>
<body>
 
 
 <div class="folium-map" id="map_d9aa7060d08aa06cbaa58663d314787a" ></div>
 
</body>
<script>
 
 
 var map_d9aa7060d08aa06cbaa58663d314787a = L.map(
 "map_d9aa7060d08aa06cbaa58663d314787a",
 {
 center: [0.0, 0.0],
 crs: L.CRS.EPSG3857,
 ...{
 "zoom": 1,
 "zoomControl": true,
 "preferCanvas": true,
}

 }
 );

 

 
 
 var tile_layer_cc9537c0c31cf7b47ce06e1d2135af6e = L.tileLayer(
 "https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
 {
 "minZoom": 0,
 "maxZoom": 20,
 "maxNativeZoom": 20,
 "noWrap": false,
 "attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"https://carto.com/attributions\"\u003eCARTO\u003c/a\u003e",
 "subdomains": "abcd",
 "detectRetina": false,
 "tms": false,
 "opacity": 1,
}

 );
 
 
 tile_layer_cc9537c0c31cf7b47ce06e1d2135af6e.addTo(map_d9aa7060d08aa06cbaa58663d314787a);
 
 
 var feature_group_3e77f122a55721e900a9958a6ca71ca9 = L.featureGroup(
 {
}
 );
 
 
 function geo_json_81de696bc8846157222da6a09f9fd155_styler(feature) {
 switch(feature.properties.route_id) {
 default:
 return {"color": "#66c2a5"};
 }
 }

 function geo_json_81de696bc8846157222da6a09f9fd155_onEachFeature(feature, layer) {

 layer.on({
 });
 };
 var geo_json_81de696bc8846157222da6a09f9fd155 = L.geoJson(null, {
 onEachFeature: geo_json_81de696bc8846157222da6a09f9fd155_onEachFeature,
 
 style: geo_json_81de696bc8846157222da6a09f9fd155_styler,
 ...{
}
 });

 function geo_json_81de696bc8846157222da6a09f9fd155_add (data) {
 geo_json_81de696bc8846157222da6a09f9fd155
 .addData(data);
 }
 geo_json_81de696bc8846157222da6a09f9fd155_add({"features": [{"geometry": {"coordinates": [[[145.678565, -16.784916], [145.678801, -16.784732], [145.679266, -16.784366]], [[145.679266, -16.784366], [145.678844, -16.784698], [145.678565, -16.784916]], [[145.68036, -16.790281], [145.679578, -16.789381], [145.679444, -16.789132], [145.679263, -16.788384], [145.679226, -16.788229], [145.679068, -16.787937], [145.678184, -16.786928], [145.678142, -16.786845], [145.678119, -16.786743], [145.678109, -16.78651], [145.678101, -16.785867], [145.67807, -16.785768], [145.678016, -16.7856

In [ ]:
# Alternatively map routes without stops using GeoPandas's explore

(
    feed.get_routes(as_geo=True)
    .collect()
    .st.to_geopandas()
    .explore(
        column="route_short_name",
        style_kwds=dict(weight=3),
        highlight_kwds=dict(weight=8),
        tiles="CartoDB positron",
    )
)

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_72e9ef8309a04b943fb91d06942ed0cb {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>

 <style>html, body {
 width: 100%;
 height: 100%;
 margin: 0;
 padding: 0;
 }
 </style>

 <style>#map {
 position:absolute;
 top:0;
 bottom:0;
 right:0;
 left:0;
 }
 </style>

 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>

 
 
 <style>
 .foliumtooltip {
 
 }
 .foliumtooltip table{
 margin: auto;
 }
 .foliumtooltip tr{
 text-align: left;
 }
 .foliumtooltip th{
 padding: 2px; padding-right: 8px;
 }
 </style>
 
 
 <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
 <script>$( function() {
 $( ".maplegend" ).draggable({
 start: function (event, ui) {
 $(this).css({
 right: "auto",
 top: "auto",
 bottom: "auto"
 });
 }
 });
 });
 </script>
 <style type='text/css'>
 .maplegend {
 position: absolute;
 z-index:9999;
 background-color: rgba(255, 255, 255, .8);
 border-radius: 5px;
 box-shadow: 0 0 15px rgba(0,0,0,0.2);
 padding: 10px;
 font: 12px/14px Arial, Helvetica, sans-serif;
 right: 10px;
 bottom: 20px;
 }
 .maplegend .legend-title {
 text-align: left;
 margin-bottom: 5px;
 font-weight: bold;
 }
 .maplegend .legend-scale ul {
 margin: 0;
 margin-bottom: 0px;
 padding: 0;
 float: left;
 list-style: none;
 }
 .maplegend .legend-scale ul li {
 list-style: none;
 margin-left: 0;
 line-height: 16px;
 margin-bottom: 2px;
 }
 .maplegend ul.legend-labels li span {
 display: block;
 float: left;
 height: 14px;
 width: 14px;
 margin-right: 5px;
 margin-left: 0;
 border: 0px solid #ccc;
 }
 .maplegend .legend-source {
 color: #777;
 clear: both;
 }
 .maplegend a {
 color: #777;
 }
 </style>
 
</head>
<body>
 
 
 <div id='maplegend route_short_name' class='maplegend'>
 <div class='legend-title'>route_short_name</div>
 <div class='legend-scale'>
 <ul class='legend-labels'>
 <li><span style='background:#1f77b4'></span>110</li>
 <li><span style='background:#1f77b4'></span>110N</li>
 <li><span style='background:#aec7e8'></span>111</li>
 <li><span style='background:#ff7f0e'></span>112</li>
 <li><span style='background:#ffbb78'></span>113</li>
 <li><span style='background:#2ca02c'></span>120</li>
 <li><span style='background:#98df8a'></span>120N</li>
 <li><span style='background:#d62728'></span>121</li>
 <li><span style='background:#ff9896'></span>122</li>
 <li><span style='background:#9467bd'></span>123</li>
 <li><span style='background:#c5b0d5'></span>130</li>
 <li><span style='background:#8c564b'></span>131</li>
 <li><span style='background:#c49c94'></span>131N</li>
 <li><span style='backgro

In [ ]:
# Show screen line

trip_id = "CNS2014-CNS_MUL-Weekday-00-4166247"
m = feed_1.map_trips([trip_id], show_stops=True, show_direction=True)
screen_line = st.read_file(DATA / "cairns_screen_line.geojson")
screen_line_gj = json.loads(screen_line.st.to_geojson().row(0)[0])
keys_to_remove = [
    key
    for key in m._children.keys()
    if key.startswith("layer_control_") or key.startswith("fit_bounds_")
]
for key in keys_to_remove:
    m._children.pop(key)
fg = fl.FeatureGroup(name="Screen lines")
fl.GeoJson(
    screen_line_gj, style_function=lambda feature: {"color": "red", "weight": 2}
).add_to(fg)
fg.add_to(m)
fl.LayerControl().add_to(m)
m.fit_bounds(fg.get_bounds())
m

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_e7d4ef23c9d65c75d44e8b39dce797d1 {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>

 <style>html, body {
 width: 100%;
 height: 100%;
 margin: 0;
 padding: 0;
 }
 </style>

 <style>#map {
 position:absolute;
 top:0;
 bottom:0;
 right:0;
 left:0;
 }
 </style>

 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>

 
 <script src="https://cdn.jsdelivr.net/npm/leaflet-textpath@1.2.3/leaflet.textpath.min.js"></script>
</head>
<body>
 
 
 <div class="folium-map" id="map_e7d4ef23c9d65c75d44e8b39dce797d1" ></div>
 
</body>
<script>
 
 
 var map_e7d4ef23c9d65c75d44e8b39dce797d1 = L.map(
 "map_e7d4ef23c9d65c75d44e8b39dce797d1",
 {
 center: [0.0, 0.0],
 crs: L.CRS.EPSG3857,
 ...{
 "zoom": 1,
 "zoomControl": true,
 "preferCanvas": false,
}

 }
 );

 

 
 
 var tile_layer_371c3f031b01a135fd48ee3e5bfc7e45 = L.tileLayer(
 "https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
 {
 "minZoom": 0,
 "maxZoom": 20,
 "maxNativeZoom": 20,
 "noWrap": false,
 "attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"https://carto.com/attributions\"\u003eCARTO\u003c/a\u003e",
 "subdomains": "abcd",
 "detectRetina": false,
 "tms": false,
 "opacity": 1,
}

 );
 
 
 tile_layer_371c3f031b01a135fd48ee3e5bfc7e45.addTo(map_e7d4ef23c9d65c75d44e8b39dce797d1);
 
 
 var feature_group_130dbaf776195d748c3f3d32c4299603 = L.featureGroup(
 {
}
 );
 
 
 var poly_line_d29d29b2bc1bce19a8e2942a78d983e8 = L.polyline(
 [[-16.835031, 145.692469], [-16.835097, 145.692414], [-16.835168, 145.692352], [-16.835139, 145.692246], [-16.835092, 145.692191], [-16.835001, 145.69218], [-16.834937, 145.692194], [-16.834894, 145.69226], [-16.834867, 145.692345], [-16.83487, 145.692515], [-16.834886, 145.692673], [-16.83489, 145.693], [-16.834891, 145.693086], [-16.83489, 145.693316], [-16.834895, 145.69374], [-16.834885, 145.693992], [-16.8348, 145.694504], [-16.834839, 145.694982], [-16.834801, 145.694986], [-16.834756, 145.695001], [-16.834714, 145.695026], [-16.834679, 145.69506], [-16.83465, 145.6951], [-16.834631, 145.695146], [-16.83462, 145.695195], [-16.83462, 145.695245], [-16.834631, 145.695294], [-16.83465, 145.69534], [-16.834679, 145.69538], [-16.834714, 145.695414], [-16.834756, 145.695439], [-16.834801, 145.695454], [-16.834849, 145.695459], [-16.834874, 145.695456], [-16.834889, 145.69567], [-16.834921, 145.696153], [-16.834638, 145.696281], [-16.834388, 145.696307], [-16.834232, 145.696287], [-16.833908, 145.696105], [

In [ ]:
# Screen line counts

slc = feed_1.compute_screen_line_counts(screen_line, dates=dates).collect()
slc.filter(pl.col("trip_id") == trip_id)

date,screen_line_id,shape_id,trip_id,direction_id,route_id,route_short_name,route_type,crossing_direction,crossing_time,crossing_dist_m
str,str,str,str,i8,str,str,i8,i32,str,f64
"""20140526""","""sl000""","""1120011""","""CNS2014-CNS_MUL-Weekday-00-416…",0,"""112-423""","""112""",3,-1,"""08:09:43""",9399.456873
"""20140526""","""sl000""","""1120011""","""CNS2014-CNS_MUL-Weekday-00-416…",0,"""112-423""","""112""",3,1,"""08:13:02""",11910.653361
